# Week2 과제 - 김진영

## Q1-1) 아래에 주어진 주석을 기반으로 하여 코딩을 해주세요.

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pylab as plt
import random

In [2]:
# 파라미터 설정 (learning rate, training epochs, batch_size)
learning_rate = 0.1
training_epochs = 15
batch_size = 100

# train과 test set으로 나누어 MNIST data 불러오기
mnist_train=dsets.MNIST(root='MNIST_data/',
                      train=True,
                      transform=transforms.ToTensor(),
                      download=True)
mnist_test=dsets.MNIST(root='MNIST_data/',
                      train=False,
                      transform=transforms.ToTensor(),
                      download=True)

# dataset loader에 train과 test할당하기
# (batch size, shuffle, drop_last 잘 설정할 것!)
train_loader = DataLoader(dataset=mnist_train,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=True)

test_loader = DataLoader(dataset=mnist_test,
                         batch_size=batch_size,
                         shuffle=False,
                         drop_last=True)


# Layer 쌓기 
# (조건: 3개의 Layer 사용, DropOut 사용 (p=0.3), ReLU 함수 사용, Batch normalization하기)
# 각 Layer의 Hidden node 수 : 1st Layer (784,100), 2nd Layer(100,100),3rd Layer(100,10)
linear1=nn.Linear(784,100,bias=True)
linear2=nn.Linear(100,100,bias=True)
linear3=nn.Linear(100,10,bias=True)
dropout=nn.Dropout(p=0.3)
relu=nn.ReLU()
bn1=nn.BatchNorm1d(100)
bn2=nn.BatchNorm1d(100)

#xavier initialization을 이용하여 각 layer의 weight 초기화
nn.init.xavier_uniform_(linear1.weight)
nn.init.xavier_uniform_(linear2.weight)
nn.init.xavier_uniform_(linear3.weight)

# torch.nn.Sequential을 이용하여 model 정의하기
# (쌓는 순서: linear-Batch Normalization Layer - ReLU- DropOut)
bn_model=torch.nn.Sequential(linear1,bn1,relu,dropout,
                         linear2,bn2,relu,dropout,
                         linear3)

# Loss Function 정의하기 (CrossEntropy를 사용할 것!)
criterion=nn.CrossEntropyLoss() # softmax는 안에 포함

#optimizer 정의하기 (Adam optimizer를 사용할 것!)
optimizer=optim.Adam(bn_model.parameters(),lr=learning_rate)

#cost 계산을 위한 변수 설정
train_total_batch = len(train_loader)

#Training epoch (cost 값 초기 설정(0으로)과 model의 train 설정 꼭 할 것) 

bn_model.train()

for epoch in range(training_epochs):
    avg_cost=0

#train dataset을 불러오고(X,Y 불러오기), back propagation과 optimizer를 사용하여 loss를 최적화하는 코드
    for X, Y in train_loader:
        X=X.view(-1,28*28)
        Y=Y
        
        hypothesis=bn_model(X)
        bn_loss=criterion(hypothesis,Y)
        
        optimizer.zero_grad()
        bn_loss.backward()
        optimizer.step()
    
        avg_cost += bn_loss / train_total_batch
        
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

#test data로 모델의 정확도를 검증하는 코드 (model의 evaluation mode 설정 꼭 할 것)
#X_test 불러올 때 view를 사용하여 차원 변환할 것/ Y_test를 불러올때 labels사용
#accuracy의 초기 값 설정(0으로) 꼭 할 것

with torch.no_grad():
    bn_model.eval() # Set the model to evaluation mode
    bn_loss, bn_acc=0,0 # loss, accuracy 초기값 0으로 설정
    
    for X, Y in test_loader:
        X=X.view(-1,28*28)
        Y=Y
        
        bn_prediction=bn_model(X)
        bn_correct_prediction=torch.argmax(bn_prediction,1)==Y
        bn_loss+=criterion(bn_prediction,Y)
        bn_acc+=bn_correct_prediction.float().mean()
    print("Accuracy: ", bn_acc.item())
    
    ##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드 
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    
    print('Label: ', Y_single_data.item())
    single_prediction = bn_model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())


Epoch: 0001 cost = 0.512296796
Epoch: 0002 cost = 0.375564367
Epoch: 0003 cost = 0.340251803
Epoch: 0004 cost = 0.310964823
Epoch: 0005 cost = 0.296591580
Epoch: 0006 cost = 0.290034473
Epoch: 0007 cost = 0.281093180
Epoch: 0008 cost = 0.275188625
Epoch: 0009 cost = 0.267442733
Epoch: 0010 cost = 0.273252875
Epoch: 0011 cost = 0.258959025
Epoch: 0012 cost = 0.248197109
Epoch: 0013 cost = 0.249155253
Epoch: 0014 cost = 0.240713373
Epoch: 0015 cost = 0.242880628
Learning finished
Accuracy:  96.67001342773438
Label:  2
Prediction:  2


C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


## Q1-2) 지금까지는 Layer의 수를 바꾸거나, Batch Normalization Layer를 추가하는 등 Layer에만 변화를 주며 모델의 성능을 향상 시켰습니다.
### 이번 문제에서는 위에서 만든 모델에서 있던 Layer 들의 Hidden node 수를 증가 또는 감소 (ex: 200, 300, 50...) 시켰을 때, train set에서의 cost와 test set에서 Accuracy가 기존 결과와 비교하였을 때 어떻게 달라졌는지 비교해주시면 됩니다.

### (1) hidden node 수 증가 (100->200)

In [3]:
# 파라미터 설정 (learning rate, training epochs, batch_size)
learning_rate = 0.1
training_epochs = 15
batch_size = 100

# train과 test set으로 나누어 MNIST data 불러오기
mnist_train=dsets.MNIST(root='MNIST_data/',
                      train=True,
                      transform=transforms.ToTensor(),
                      download=True)
mnist_test=dsets.MNIST(root='MNIST_data/',
                      train=False,
                      transform=transforms.ToTensor(),
                      download=True)

# dataset loader에 train과 test할당하기
# (batch size, shuffle, drop_last 잘 설정할 것!)
train_loader = DataLoader(dataset=mnist_train,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=True)

test_loader = DataLoader(dataset=mnist_test,
                         batch_size=batch_size,
                         shuffle=False,
                         drop_last=True)


# Layer 쌓기 
# (조건: 3개의 Layer 사용, DropOut 사용 (p=0.3), ReLU 함수 사용, Batch normalization하기)
# 각 Layer의 Hidden node 수 : 1st Layer (784,200), 2nd Layer(200,200),3rd Layer(200,10)
linear1=nn.Linear(784,200,bias=True)
linear2=nn.Linear(200,200,bias=True)
linear3=nn.Linear(200,10,bias=True)
dropout=nn.Dropout(p=0.3)
relu=nn.ReLU()
bn1=nn.BatchNorm1d(200)
bn2=nn.BatchNorm1d(200)

#xavier initialization을 이용하여 각 layer의 weight 초기화
nn.init.xavier_uniform_(linear1.weight)
nn.init.xavier_uniform_(linear2.weight)
nn.init.xavier_uniform_(linear3.weight)

# torch.nn.Sequential을 이용하여 model 정의하기
# (쌓는 순서: linear-Batch Normalization Layer - ReLU- DropOut)
bn_model=torch.nn.Sequential(linear1,bn1,relu,dropout,
                         linear2,bn2,relu,dropout,
                         linear3)

# Loss Function 정의하기 (CrossEntropy를 사용할 것!)
criterion=nn.CrossEntropyLoss() # softmax는 안에 포함

#optimizer 정의하기 (Adam optimizer를 사용할 것!)
optimizer=optim.Adam(bn_model.parameters(),lr=learning_rate)

#cost 계산을 위한 변수 설정
train_total_batch = len(train_loader)

#Training epoch (cost 값 초기 설정(0으로)과 model의 train 설정 꼭 할 것) 

bn_model.train()

for epoch in range(training_epochs):
    avg_cost=0

#train dataset을 불러오고(X,Y 불러오기), back propagation과 optimizer를 사용하여 loss를 최적화하는 코드
    for X, Y in train_loader:
        X=X.view(-1,28*28)
        Y=Y
        
        hypothesis=bn_model(X)
        bn_loss=criterion(hypothesis,Y)
        
        optimizer.zero_grad()
        bn_loss.backward()
        optimizer.step()
    
        avg_cost += bn_loss / train_total_batch
        
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

#test data로 모델의 정확도를 검증하는 코드 (model의 evaluation mode 설정 꼭 할 것)
#X_test 불러올 때 view를 사용하여 차원 변환할 것/ Y_test를 불러올때 labels사용
#accuracy의 초기 값 설정(0으로) 꼭 할 것

with torch.no_grad():
    bn_model.eval() # Set the model to evaluation mode
    bn_loss, bn_acc=0,0 # loss, accuracy 초기값 0으로 설정
    
    for X, Y in test_loader:
        X=X.view(-1,28*28)
        Y=Y
        
        bn_prediction=bn_model(X)
        bn_correct_prediction=torch.argmax(bn_prediction,1)==Y
        bn_loss+=criterion(bn_prediction,Y)
        bn_acc+=bn_correct_prediction.float().mean()
    print("Accuracy: ", bn_acc.item())
    
    ##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드 
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    
    print('Label: ', Y_single_data.item())
    single_prediction = bn_model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())


Epoch: 0001 cost = 0.477384955
Epoch: 0002 cost = 0.331772596
Epoch: 0003 cost = 0.292640299
Epoch: 0004 cost = 0.278033406
Epoch: 0005 cost = 0.262251168
Epoch: 0006 cost = 0.256022990
Epoch: 0007 cost = 0.239788041
Epoch: 0008 cost = 0.236049607
Epoch: 0009 cost = 0.228607759
Epoch: 0010 cost = 0.222600222
Epoch: 0011 cost = 0.216132000
Epoch: 0012 cost = 0.213803470
Epoch: 0013 cost = 0.204192325
Epoch: 0014 cost = 0.210139528
Epoch: 0015 cost = 0.198376372
Learning finished
Accuracy:  97.34000396728516
Label:  9
Prediction:  4


#### accuracy가 조금 증가하였다.

### (2) hidden node 수 감소 (100->50)

In [4]:
# 파라미터 설정 (learning rate, training epochs, batch_size)
learning_rate = 0.1
training_epochs = 15
batch_size = 100

# train과 test set으로 나누어 MNIST data 불러오기
mnist_train=dsets.MNIST(root='MNIST_data/',
                      train=True,
                      transform=transforms.ToTensor(),
                      download=True)
mnist_test=dsets.MNIST(root='MNIST_data/',
                      train=False,
                      transform=transforms.ToTensor(),
                      download=True)

# dataset loader에 train과 test할당하기
# (batch size, shuffle, drop_last 잘 설정할 것!)
train_loader = DataLoader(dataset=mnist_train,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=True)

test_loader = DataLoader(dataset=mnist_test,
                         batch_size=batch_size,
                         shuffle=False,
                         drop_last=True)


# Layer 쌓기 
# (조건: 3개의 Layer 사용, DropOut 사용 (p=0.3), ReLU 함수 사용, Batch normalization하기)
# 각 Layer의 Hidden node 수 : 1st Layer (784,50), 2nd Layer(50,50),3rd Layer(50,10)
linear1=nn.Linear(784,50,bias=True)
linear2=nn.Linear(50,50,bias=True)
linear3=nn.Linear(50,10,bias=True)
dropout=nn.Dropout(p=0.3)
relu=nn.ReLU()
bn1=nn.BatchNorm1d(50)
bn2=nn.BatchNorm1d(50)

#xavier initialization을 이용하여 각 layer의 weight 초기화
nn.init.xavier_uniform_(linear1.weight)
nn.init.xavier_uniform_(linear2.weight)
nn.init.xavier_uniform_(linear3.weight)

# torch.nn.Sequential을 이용하여 model 정의하기
# (쌓는 순서: linear-Batch Normalization Layer - ReLU- DropOut)
bn_model=torch.nn.Sequential(linear1,bn1,relu,dropout,
                         linear2,bn2,relu,dropout,
                         linear3)

# Loss Function 정의하기 (CrossEntropy를 사용할 것!)
criterion=nn.CrossEntropyLoss() # softmax는 안에 포함

#optimizer 정의하기 (Adam optimizer를 사용할 것!)
optimizer=optim.Adam(bn_model.parameters(),lr=learning_rate)

#cost 계산을 위한 변수 설정
train_total_batch = len(train_loader)

#Training epoch (cost 값 초기 설정(0으로)과 model의 train 설정 꼭 할 것) 

bn_model.train()

for epoch in range(training_epochs):
    avg_cost=0

#train dataset을 불러오고(X,Y 불러오기), back propagation과 optimizer를 사용하여 loss를 최적화하는 코드
    for X, Y in train_loader:
        X=X.view(-1,28*28)
        Y=Y
        
        hypothesis=bn_model(X)
        bn_loss=criterion(hypothesis,Y)
        
        optimizer.zero_grad()
        bn_loss.backward()
        optimizer.step()
    
        avg_cost += bn_loss / train_total_batch
        
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

#test data로 모델의 정확도를 검증하는 코드 (model의 evaluation mode 설정 꼭 할 것)
#X_test 불러올 때 view를 사용하여 차원 변환할 것/ Y_test를 불러올때 labels사용
#accuracy의 초기 값 설정(0으로) 꼭 할 것

with torch.no_grad():
    bn_model.eval() # Set the model to evaluation mode
    bn_loss, bn_acc=0,0 # loss, accuracy 초기값 0으로 설정
    
    for X, Y in test_loader:
        X=X.view(-1,28*28)
        Y=Y
        
        bn_prediction=bn_model(X)
        bn_correct_prediction=torch.argmax(bn_prediction,1)==Y
        bn_loss+=criterion(bn_prediction,Y)
        bn_acc+=bn_correct_prediction.float().mean()
    print("Accuracy: ", bn_acc.item())
    
    ##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드 
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    
    print('Label: ', Y_single_data.item())
    single_prediction = bn_model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())


Epoch: 0001 cost = 0.586803973
Epoch: 0002 cost = 0.439989895
Epoch: 0003 cost = 0.402204126
Epoch: 0004 cost = 0.393706650
Epoch: 0005 cost = 0.379411131
Epoch: 0006 cost = 0.377140313
Epoch: 0007 cost = 0.359739065
Epoch: 0008 cost = 0.361200511
Epoch: 0009 cost = 0.351111501
Epoch: 0010 cost = 0.335307926
Epoch: 0011 cost = 0.340453863
Epoch: 0012 cost = 0.330862135
Epoch: 0013 cost = 0.334065765
Epoch: 0014 cost = 0.328688413
Epoch: 0015 cost = 0.327466607
Learning finished
Accuracy:  96.17997741699219
Label:  5
Prediction:  5


#### accuracy가 조금 감소하였다.